In [118]:
%pip install sklearn


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


1. Скачиваем данные путем парсинга

Я буду использовать отзывы с сайта Steam на игру Hitman: Absolution
Сначала в фильтрах ставим язык: английский, отзывы: только положительные.
Так как мы имеем дело не с несколькими страницами, а со скроллингом вниз, то используем блок selenium

In [1]:
%pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 6.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install chromedriver-autoinstaller


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [136]:
from sklearn.metrics import accuracy_score
from nltk.util import ngrams

In [21]:
import urllib.request
from urllib.request import urlopen
import ssl
import json
ssl._create_default_https_context = ssl._create_unverified_context

In [22]:
import time
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

CHROME >= 115.0.5763.0, using mac-arm64 as architecture identifier


'/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/chromedriver_autoinstaller/117/chromedriver'

In [123]:
def get_reviews(url, num):

    # set up the webdriver
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    last_position = driver.execute_script('return window.pageYOffset;')
    reviews = []
    review_ids = set()

    while True:
      cards = driver.find_elements(By.CLASS_NAME, 'apphub_Card')
      if(len(cards) >= num):
          break
      last_position = driver.execute_script('return window.pageYOffset;')
      driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
      curr_position = driver.execute_script('return window.pageYOffset;')
      if(last_position == curr_position):
          continue

    reviews = []
    for card in cards[:num]:
        date_posted = card.find_element(By.XPATH, './/div[@class="apphub_CardTextContent"]/div').text
        review_content = card.find_element(By.XPATH, './/div[@class="apphub_CardTextContent"]').text.replace(date_posted, '').strip()
        reviews.append(review_content)

    driver.quit()

    return reviews

Покажу, что работает на 40 на каждый тип отзывов.

In [105]:
positive_reviews = get_reviews('https://steamcommunity.com/app/203140/positivereviews/?filterLanguage=english&p=1&browsefilter=toprated.html', 40)

In [106]:
negative_reviews = get_reviews('https://steamcommunity.com/app/203140/negativereviews/?filterLanguage=english&p=1&browsefilter=toprated.html', 40)

Смотрим, что отзывы реально сохранились

In [107]:
positive_reviews[10]

'Stealth is optional'

In [108]:
negative_reviews[3]

'Contracts Servers are down ( i dont care if its Io-Interactive or square enix fault, lack of interest or whatever )\nAbout 50 % of the game is useless now. Contracts Mode is still advertised as a Feature of the Game, so are the unachievable Achievements'

Делим данные на обучающие и тестовые.

In [109]:
test = positive_reviews[0:10] + negative_reviews[0:10]
positive_reviews = positive_reviews[-30:]
negative_reviews = negative_reviews[-30:]

Переходим к токенизации, лемматизации.

In [110]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nastyapakhomova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nastyapakhomova/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nastyapakhomova/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nastyapakhomova/nltk_data...


True

Пишем функцию, которая будет определять части речи, лемматизировать и токенизировать текст, а также выкидывать токены, в которых нет букв алфавита (например, смайлики, скобки и тд.)

In [111]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import re

lmtzr = WordNetLemmatizer()
def lemmatize_text(text):
    tokenized = [w for w in word_tokenize(text.lower()) if bool(re.search('[a-z]', w))]
    return [lmtzr.lemmatize(i,j[0].lower())
        if j[0].lower() in ['a', 'n', 'v']
        else lmtzr.lemmatize(i)
        for i,j in pos_tag(tokenized)]

Теперь, когда все слова приведены в порядок, нужно создать словари частнотности, в которых будут слова, которые наиболее часто встречаются в положительных или отрицательных отзывах.

In [112]:
from collections import Counter

In [113]:
def collect_freqlist(reviews):
    freqlist = Counter()
    for review in reviews:
        for token in lemmatize_text(review): #параллельно применяем нашу функцию по лемматизации
            freqlist[token] += 1
    return freqlist

In [114]:
def prepare_freqlists(positive_reviews, negative_reviews):
    positive_freqlist = collect_freqlist(positive_reviews)
    negative_freqlist = collect_freqlist(negative_reviews)
    positive_freqlist = set(k for k, v in positive_freqlist.items() if v > 2) #убираем слова, которые встречаются более двух раз
    negative_freqlist = set(k for k, v in negative_freqlist.items() if v > 2)
    positive_freqlist_unique = positive_freqlist - negative_freqlist #словарь с положителльными словами
    negative_freqlist_unique = negative_freqlist - positive_freqlist #словарь с отрицательными словами
    return positive_freqlist_unique, negative_freqlist_unique

In [115]:
positive_freqlist_unique, negative_freqlist_unique = prepare_freqlists(positive_reviews, negative_reviews)

После того, как мы создали словари, нужно сделать функцию, которая уже и будет на их основе проверять отзыв-положительный (обозначим за 1) он или отрицательный (обозначим за 0)

In [116]:
def simple_sentiment_analysis(positive_freqlist, negative_freqlist, test):
    prediction = []
    for review in test:
        positive_count = 0
        negative_count = 0
        for token in lemmatize_text(review):
            if token in positive_freqlist:
                positive_count += 1
            elif token in negative_freqlist:
                negative_count += 1
        if positive_count >= negative_count:
            prediction.append(1)
        else:
            prediction.append(0)
    return prediction

In [117]:
correct_answers = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [119]:
prediction = simple_sentiment_analysis(positive_freqlist_unique, negative_freqlist_unique, test)

Предсказываем ответы и считаем точность всего

In [121]:
print('Accuracy: %.4f' % accuracy_score(prediction, correct_answers))

Accuracy: 0.5000


Accuracy: 0.5000 результат ни туда ни сюда, поэтому сейчас предложу варианты, как можно улучшить код:
    1) Увеличить выборку конечно же. Это даст нам больше контекстов и слов, которые могут встречаться только в положительных и не встречаются в отрицательных.
    2) Также можно попробовать учитывать n-gramm (биграммы) вместо слов, возможно это повысит точность. 

1) Способ

In [126]:
posit_reviews = get_reviews('https://steamcommunity.com/app/203140/positivereviews/?filterLanguage=english&p=1&browsefilter=toprated.html', 70)

In [132]:
negat_reviews = get_reviews('https://steamcommunity.com/app/203140/negativereviews/?filterLanguage=english&p=1&browsefilter=toprated.html', 70)

In [133]:
test_2 = posit_reviews[0:10] + negat_reviews[0:10]
posit_reviews = posit_reviews[-60:]
negat_reviews = negat_reviews[-60:]

In [134]:
positive_freqlist_unique, negative_freqlist_unique = prepare_freqlists(posit_reviews, negat_reviews)

In [135]:
prediction = simple_sentiment_analysis(positive_freqlist_unique, negative_freqlist_unique, test_2)
print("Accuracy: %.4f" % accuracy_score(prediction, correct_answers))

Accuracy: 0.5500


Точность повысилась конечно не на много, но все же! Если брать выборку по 300 примеров (а не как я на 30 больше чем в первом случае) на каждый тип отзывов, то думаю точность повыситься в разы

2) Способ

In [137]:
def collect_freqlist(reviews):
    freqlist = Counter()
    for review in reviews:
        lemmatized = lemmatize_text(review)
        for token in lemmatized:
            freqlist[token] += 1
        n_grams = ngrams(lemmatized, 2)
        n_grams = [' '.join(grams) for grams in n_grams]
        for n_gram in n_grams:
            freqlist[n_gram] += 1
    return freqlist

In [138]:
positive_freqlist_unique, negative_freqlist_unique = prepare_freqlists(posit_reviews, negat_reviews)

In [139]:
prediction = simple_sentiment_analysis(positive_freqlist_unique, negative_freqlist_unique, test_2)
print("Accuracy: %.4f" % accuracy_score(prediction, correct_answers))

Accuracy: 0.5500


Точность также по сравнению с основной работой повысилась, но осталось такой же как в способе с увеличением выборки.